# Faire des prédictions 

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/SpeechRecognition')
!pip install soundfile
!pip install tf-nightly
!pip install tensorflow-gpu

     |████████████████████████████████| 390.7MB 38kB/s 
     |████████████████████████████████| 10.2MB 40.6MB/s 
     |████████████████████████████████| 460kB 46.4MB/s 
     |████████████████████████████████| 320.4MB 52kB/s 


In [3]:
sys.path.append('/content/drive/My Drive/Colab Notebooks/SpeechRecognition')

import tensorflow as tf
#import Preprocess_SpeechReco as old_data
import Preprocess_bigdataset as data 
import os 
from tensorflow.keras.models import Sequential, Model , load_model
import numpy as np
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import time 
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D


In [4]:
x_train,y_train,x_test,y_test,dic_voca=data.getData(mfccs=True,split=True)


dict index   {'@': 0, '_': 1, '"': 2, 'w': 3, 'a': 4, 'r': 5, 'j': 6, '~': 7, 'H': 8, ' ': 9, 'A': 10, 'l': 11, 't': 12, 'q': 13, 'o': 14, 'i': 15, 'y': 16, 'u': 17, '*': 18, '>': 19, 'E': 20, 'd': 21, 'h': 22, 'm': 23, 'b': 24, '^': 25, 'D': 26, 'p': 27, 'f': 28, 'k': 29, 'S': 30, 'n': 31, '-': 32, 's': 33, 'T': 34, '<': 35, '&': 36, 'Y': 37, '$': 38, 'x': 39, 'K': 40, '|': 41, '}': 42, 'g': 43, 'z': 44, "'": 45, 'F': 46, 'Z': 47, 'N': 48, '.': 49, 'v': 50}


In [5]:
x_train_no_mfcc,_,x_test_no_mfcc,_,_=data.getData(mfccs=False,split=True)

dict index   {'@': 0, '_': 1, '"': 2, 'w': 3, 'a': 4, 'r': 5, 'j': 6, '~': 7, 'H': 8, ' ': 9, 'A': 10, 'l': 11, 't': 12, 'q': 13, 'o': 14, 'i': 15, 'y': 16, 'u': 17, '*': 18, '>': 19, 'E': 20, 'd': 21, 'h': 22, 'm': 23, 'b': 24, '^': 25, 'D': 26, 'p': 27, 'f': 28, 'k': 29, 'S': 30, 'n': 31, '-': 32, 's': 33, 'T': 34, '<': 35, '&': 36, 'Y': 37, '$': 38, 'x': 39, 'K': 40, '|': 41, '}': 42, 'g': 43, 'z': 44, "'": 45, 'F': 46, 'Z': 47, 'N': 48, '.': 49, 'v': 50}


In [6]:
import numpy as np


def beam_search(model,  k, src_input, sequence_max_len,dict_voc,mfcc):
    # (log(1), initialize_of_zeros)
    if mfcc: 
      src_input=np.reshape(src_input, (1,src_input.shape[0],src_input.shape[1]) )

    else : 
      src_input=np.reshape(src_input, (1,src_input.shape[0]) )
    k_beam = [(0, [float(dict_voc['@'])]*(sequence_max_len))]
    print(k_beam)
    
    # l : point on target sentence to predict
    for l in range(sequence_max_len):
        all_k_beams = []
        for prob, sent_predict in k_beam:
            predicted = model.predict([src_input, np.array([sent_predict])])[0]
            #print(predicted)
            # top k!
            possible_k = predicted[l].argsort()[-k:][::-1]
            #print(possible_k)
            # add to all possible candidates for k-beams
            all_k_beams += [
                (
                    sum(np.log(predicted[i][sent_predict[i+1]]) for i in range(l)) + np.log(predicted[l][next_wid]),
                    list(sent_predict[:l+1])+[next_wid]+[0]*(sequence_max_len-l-2)
                )
                for next_wid in possible_k
            ]
        # top k
        k_beam = sorted(all_k_beams)[-k:]

    return k_beam



'#/content/drive/My Drive/Colab Notebooks/SpeechRecognition/fitted_moddels/Modele_SpeechReco_1598460579_[0.9320278167724609, 0.7420995831489563].h5\n#/content/drive/My Drive/Colab Notebooks/SpeechRecognition/fitted_moddels/Modele_SpeechReco_1598532411_[0.3269703686237335, 0.9057465195655823].h5\nmodel= load_model("/content/drive/My Drive/Colab Notebooks/SpeechRecognition/fitted_moddels/Modele_SpeechReco_1598612777_[0.3621765375137329, 0.8927567601203918].h5", \n                  custom_objects={\'Melspectrogram\':Melspectrogram,\n                        \'Normalization2D\':Normalization2D})\noutput_gre1 =beam_search(model,32,x_test[60],y_test.shape[1]-1, dic_voca)\nprint(output_gre1)'

In [9]:
%matplotlib inline  
import matplotlib
import matplotlib.pyplot as plt
import seaborn



def attent_and_generate( preprocessd_input_audio,dict_voc , OUTPUT_LENGTH):
    seaborn.set(font=['Osaka'], font_scale=3)

    preprocessd_input_audio=np.reshape(preprocessd_input_audio, (1,preprocessd_input_audio.shape[0]) )
    print(preprocessd_input_audio[0][100])
    START_CHAR_CODE = dict_voc['@']
    decoder_input = np.zeros(shape=(len(preprocessd_input_audio), OUTPUT_LENGTH))
    decoder_input[:,0] = START_CHAR_CODE
   
    for i in range(1, OUTPUT_LENGTH):
        
        #argmax donne l'indice qui a la plus grande proba !
        output, attention=attention_model.predict([preprocessd_input_audio, decoder_input])
        output = output.argmax(axis=2)        
        decoder_input[:,i] = output[:,i]
        attention_density = attention[0]
        decoded_output = data.Decode_sentence(decoder_input[0][1:],"DICT",dict_voc)
        
    return decoder_input[:,1:], decoded_output

def visualize(text,dict_voc , OUTPUT_LENGTH):
    model= load_model("/content/drive/My Drive/Colab Notebooks/SpeechRecognition/fitted_moddels/Modele_SpeechReco_1598612777_[0.3621765375137329, 0.8927567601203918].h5", 
                      custom_objects={'Melspectrogram':Melspectrogram,
                            'Normalization2D':Normalization2D})
    attention_layer = model.get_layer("tf_op_layer_concat_8") # or model.get_layer("tf_op_layer_concat_8")
    attention_model = Model(inputs=model.inputs, outputs=model.outputs + [attention_layer.output])
    attention_model.summary()
    attention_density, katakana = attent_and_generate(text,dict_voc , OUTPUT_LENGTH)
    print("visualize chart " , katakana)
    plt.clf()
    plt.figure(figsize=(148,120))

    ax = seaborn.heatmap(attention_density[:len(katakana) + 2, : len(text) + 2],
        xticklabels=[w for w in range(0,12)],
        yticklabels=[w for w in katakana])

    ax.invert_yaxis()
    plt.show()
#visualize(x_train[60],dic_voca,y_train.shape[1]-1)



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


int_axis=1 passed but is ignored, str_axis is used instead.
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 264600)]     0                                            
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 1, 264600)    0           input_3[0][0]                    
__________________________________________________________________________________________________
mel_stft (Melspectrogram)       (None, 80, 2068, 1)  1091664     reshape_2[0][0]                  
__________________________________________________________________________________________________
mel_stft_norm (Normalization2D) (None, 80, 2068, 1)  0           mel_stft[0][0]                   
___________________________

In [ ]:
def predict(k, path_model , path_audio,tokenzer_train , OUTPUT_LENGTH,mfccs): 
    
    #get in the model  
    model= load_model(path_model, 
                  custom_objects={'Melspectrogram':Melspectrogram,
                        'Normalization2D':Normalization2D})
    
    #preprocessd_input_audio=preprocess_new_audio(path_audio,mfcc=mfccs)[0]
    preprocessd_input_audio=path_audio
    print(preprocessd_input_audio.shape)
    #preprocessd_input_audio=path_audio
    ''' if k==1:
        predictions = Greedy_search(model, preprocessd_input_audio,tokenzer_train , OUTPUT_LENGTH)'''
  
    predictions =beam_search(model,k,preprocessd_input_audio,OUTPUT_LENGTH, dic_voca,mfcc=mfccs)
    print(predictions)
        
    return predictions

def preprocess_new_audio(path_audio="/content/drive/My Drive/Colab Notebooks/SpeechRecognition/s8L2.wav",max_duration=12,mfcc=False) :

  audio,sr= data.rosa.load(path_audio)
  print(sr)
  dur_audio=data.rosa.get_duration(audio)
  audios=[]

  if dur_audio<=max_duration : 
    #si inférieur à la durée le pad 
    pad_ms=(max_duration-dur_audio) *sr
    print("old duration ", dur_audio )
    audio=np.append(audio,np.zeros(int(round(pad_ms)))) 
    data.Plot_audio(audio,sr)

    if mfcc : 
        audio=data.build_mfccs([audio])[0]
    audios.append(audio)
  else : 
    
    # extraire les sous audios 
    nonMuteSections =data.rosa.effects.split(audio,60)
    for i in range(0,len(nonMuteSections)):  
      s=audio[nonMuteSections[i][0]:nonMuteSections[i][1]]
      if mfcc : 
        s=data.build_mfccs([s])
      audios.append(s)  
  return audios    



#path_model="/content/drive/My Drive/Colab Notebooks/SpeechRecognition/fitted_moddels/Modele_SpeechReco_1598612777_[0.3621765375137329, 0.8927567601203918].h5" 
path_model="/content/drive/My Drive/Colab Notebooks/SpeechRecognition/fitted_moddels/Modele_SpeechReco_1599299782_[0.3388335406780243, 0.9021098017692566].h5"
#path_model="/content/drive/My Drive/Colab Notebooks/SpeechRecognition/fitted_moddels/Modele_SpeechReco_1600302374_[0.35865363478660583, 0.896904468536377].h5"
#path_audio="/content/drive/My Drive/Colab Notebooks/SpeechRecognition/output.wav"
index=754   
k=64
predictions=predict(k ,path_model,x_test[index],None,y_test.shape[1]-1,True)


(517, 13)
[(0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log


In [ ]:
def viewPredictions(text_true=u' ' , predictions=[],k=k):

  for el in range(0,k) : 
    predicted = data.Decode_sentence(predictions[el][1],"DICT",dic_voca)
    proba=predictions[el][0]
    
    print(" output beam :  " ,proba,"\n ", predicted , ' \n', data.Test_Transcription( predicted[1] , False ))

  true =data.Decode_sentence(y_test[index],"DICT",dic_voca)
  print(" le true ", true)   
  true =data.Test_Transcription( true[1] , False )
  print(" le true ", true) 
data.Plot_audio(x_train_no_mfcc[index],22050)
viewPredictions(text_true="العالمية",predictions=predictions)
